In [2]:
import t3f
import numpy as np
import tensorflow as tf
import keras.backend as K

tf.set_random_seed(0)
np.random.seed(0)
sess = tf.InteractiveSession()
K.set_session(sess)

## Tensor Nets

In this notebook we provide an example of how to build a simple Tensor Net (see https://arxiv.org/abs/1509.06569).

The main ingredient is the so-called TT-Matrix, a generalization of the Kronecker product matrices, i.e. matrices of the form 
$$A = A_1 \otimes A_2 \cdots \otimes A_n$$

In `t3f` TT-Matrices are represented using the `TensorTrain` class.

In [3]:
W = t3f.random_matrix([[4, 7, 4, 7], [5, 5, 5, 5]], tt_rank=2)

print(W)

A TT-Matrix of size 784 x 625, underlying tensor shape: (4, 7, 4, 7) x (5, 5, 5, 5), TT-ranks: (1, 2, 2, 2, 1)


Using TT-Matrices we can compactly represent densely connected layers in neural networks, which allows us to greatly reduce number of parameters. Matrix multiplication can be handled by the `t3f.matmul` method which allows for multiplying dense (ordinary) matrices and TT-Matrices. Very simple neural network could look as following (for initialization several options such as `t3f.initializers.glorot`, `t3f.initializers.he` or `t3f.random_matrix` are available):

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.int64, [None])

initializer = t3f.glorot_initializer([[4, 7, 4, 7], [5, 5, 5, 5]], tt_rank=2)
W1 = t3f.get_variable('W1', initializer=initializer) 
b1 = tf.get_variable('b1', shape=[625])
h1 = t3f.matmul(x, W1) + b1
h1 = tf.nn.relu(h1)

W2 = tf.get_variable('W2', shape=[625, 10])
b2 = tf.get_variable('b2', shape=[10])
h2 = tf.matmul(h1, W2) + b2

y_ = tf.one_hot(y, 10)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=h2))

For convenience we have implemented a layer analogous to *Keras* `Dense` layer but with a TT-Matrix instead of an ordinary matrix. An example of fully trainable net is provided below.

In [5]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from utils import TTDense

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Some preprocessing...

In [7]:
x_train = x_train / 127.5 - 1.0
x_test = x_test / 127.5 - 1.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [8]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(TTDense(row_dims=[7, 4, 7, 4], column_dims=[5, 5, 5, 5], tt_rank=4, init='glorot', activation='relu', bias_init=1e-3))
model.add(Dense(10))
model.add(Activation('softmax'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
tt_dense_1 (TTDense)         (None, 625)               1725      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6260      
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 7,985
Trainable params: 7,985
Non-trainable params: 0
_________________________________________________________________


Note that in the dense layer we only have $1725$ parameters instead of $784 * 625 = 490000$.

In [10]:
optimizer = optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(x_train, y_train, epochs=2, batch_size=64, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 30s 494us/step - loss: 0.2540 - acc: 0.9233 - val_loss: 0.1394 - val_acc: 0.9569
Epoch 2/2
60000/60000 [==============================] - 28s 474us/step - loss: 0.1483 - acc: 0.9564 - val_loss: 0.1182 - val_acc: 0.9645


# Compression of Dense layers

Let us now train an ordinary DNN (without TT-Matrices) and show how we can compress it using the TT decomposition.

In [25]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(625, activation='relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 625)               490625    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                6260      
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
Total params: 496,885
Trainable params: 496,885
Non-trainable params: 0
_________________________________________________________________


In [27]:
optimizer = optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 7s 111us/step - loss: 0.2937 - acc: 0.9115 - val_loss: 0.1642 - val_acc: 0.9488
Epoch 2/5
60000/60000 [==============================] - 6s 102us/step - loss: 0.1353 - acc: 0.9590 - val_loss: 0.1297 - val_acc: 0.9606
Epoch 3/5
60000/60000 [==============================] - 6s 97us/step - loss: 0.1027 - acc: 0.9684 - val_loss: 0.0962 - val_acc: 0.9706
Epoch 4/5
60000/60000 [==============================] - 6s 100us/step - loss: 0.0813 - acc: 0.9744 - val_loss: 0.1335 - val_acc: 0.9593
Epoch 5/5
60000/60000 [==============================] - 6s 100us/step - loss: 0.0736 - acc: 0.9767 - val_loss: 0.1193 - val_acc: 0.9608


Let us convert the matrix used in the Dense layer to the TT-Matrix with tt-ranks equal to 16 (since we trained the network without the low-rank structure assumption we may wish start with high rank values).

In [29]:
W = model.trainable_weights[0]
print(W)
Wtt = t3f.to_tt_matrix(W, shape=[[7, 4, 7, 4], [5, 5, 5, 5]], max_tt_rank=16)
print(Wtt)

<tf.Variable 'dense_6/kernel:0' shape=(784, 625) dtype=float32_ref>
A TT-Matrix of size 784 x 625, underlying tensor shape: (7, 4, 7, 4) x (5, 5, 5, 5), TT-ranks: (1, 16, 16, 16, 1)


We need to evaluate the tt-cores of Wtt. We also need to store other parameters for later (biases and the second dense layer).

In [30]:
cores = sess.run(Wtt.tt_cores)
other_params = model.get_weights()[1:]

Now we can construct a tensor network with the first Dense layer replaced by `Wtt`
initialized using the previously computed cores.

In [31]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(TTDense(row_dims=[7, 4, 7, 4], column_dims=[5, 5, 5, 5], tt_rank=16, activation='relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

In [32]:
optimizer = optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.set_weights(list(cores) + other_params)

In [34]:
print("new accuracy: ", model.evaluate(x_test, y_test)[1])

10000/10000 [==============================] - 10s 972us/step
new accuracy:  0.6459


In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
tt_dense_4 (TTDense)         (None, 625)               15585     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                6260      
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
Total params: 21,845
Trainable params: 21,845
Non-trainable params: 0
_________________________________________________________________


We see that even though we now have about 5% of the original number of parameters we still achieve a relatively high accuracy.

## Finetuning the model. 

We can now finetune this tensor network.

In [36]:
model.fit(x_train, y_train, epochs=2, batch_size=64, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 117s 2ms/step - loss: 0.1346 - acc: 0.9595 - val_loss: 0.1142 - val_acc: 0.9653
Epoch 2/2
60000/60000 [==============================] - 128s 2ms/step - loss: 0.0806 - acc: 0.9759 - val_loss: 0.0823 - val_acc: 0.9737


We see that we were able to achieve higher validation accuracy than we had in the plain DNN, while keeping the number of parameters extremely small.